<a href="https://colab.research.google.com/github/prof-eduardo-nunes/unicamp-mineracao_dados/blob/main/lista_de_recomenda%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================================
# Sistema de Recomendação de Filmes
# Prof. Eduardo Nunes - FA-084 Mineração de Dados
# Adaptado do código original de Jones Granatyr
# ==========================================================

#===========================================================
# resumo das funções

#euclidiana → Calcula similaridade entre dois usuários/itens.

#getSimilares → Lista os mais parecidos com um usuário/item.

#getRecomendacoesUsuario → Recomenda filmes com base em usuários semelhantes.

#calculaItensSimilares → Cria tabela de similaridade entre itens.

#getRecomendacoesItens → Recomenda filmes com base em itens semelhantes.

#count_evaluations → Conta quantas avaliações cada filme recebeu.

#=============================================================


# Dicionário com avaliações feitas por usuários
# Formato: { "Usuário": { "Filme": nota, ... }, ... }
avaliacoesUsuario = {
    'Ana': {
        'Freddy x Jason': 2.5,
        'O Ultimato Bourne': 3.5,
        'Star Trek': 3.0,
        'Exterminador do Futuro': 3.5,
        'Norbit': 2.5,
        'Star Wars': 3.0
    },
    'Marcos': {
        'Freddy x Jason': 3.0,
        'O Ultimato Bourne': 3.5,
        'Star Trek': 1.5,
        'Exterminador do Futuro': 5.0,
        'Star Wars': 3.0,
        'Norbit': 3.5
    },
    'Pedro': {
        'Freddy x Jason': 2.5,
        'O Ultimato Bourne': 3.0,
        'Exterminador do Futuro': 3.5,
        'Star Wars': 4.0
    },
    'Claudia': {
        'O Ultimato Bourne': 3.5,
        'Star Trek': 3.0,
        'Star Wars': 4.5,
        'Exterminador do Futuro': 4.0,
        'Norbit': 2.5
    },
    'Adriano': {
        'Freddy x Jason': 3.0,
        'O Ultimato Bourne': 4.0,
        'Star Trek': 2.0,
        'Exterminador do Futuro': 3.0,
        'Star Wars': 3.0,
        'Norbit': 2.0
    },
    'Janaina': {
        'Freddy x Jason': 3.0,
        'O Ultimato Bourne': 4.0,
        'Star Wars': 3.0,
        'Exterminador do Futuro': 5.0,
        'Norbit': 3.5
    },
    'Leonardo': {
        'O Ultimato Bourne': 4.5,
        'Norbit': 1.0,
        'Exterminador do Futuro': 4.0
    }
}

# Dicionário com avaliações organizadas por filme
# Formato: { "Filme": { "Usuário": nota, ... }, ... }
avaliacoesFilme = {
    'Freddy x Jason': {
        'Ana': 2.5,
        'Marcos': 3.0,
        'Pedro': 2.5,
        'Adriano': 3.0,
        'Janaina': 3.0
    },
    'O Ultimato Bourne': {
        'Ana': 3.5,
        'Marcos': 3.5,
        'Pedro': 3.0,
        'Claudia': 3.5,
        'Adriano': 4.0,
        'Janaina': 4.0,
        'Leonardo': 4.5
    },
    'Star Trek': {
        'Ana': 3.0,
        'Marcos': 1.5,
        'Claudia': 3.0,
        'Adriano': 2.0
    },
    'Exterminador do Futuro': {
        'Ana': 3.5,
        'Marcos': 5.0,
        'Pedro': 3.5,
        'Claudia': 4.0,
        'Adriano': 3.0,
        'Janaina': 5.0,
        'Leonardo': 4.0
    },
    'Norbit': {
        'Ana': 2.5,
        'Marcos': 3.0,
        'Claudia': 2.5,
        'Adriano': 2.0,
        'Janaina': 3.5,
        'Leonardo': 1.0
    },
    'Star Wars': {
        'Ana': 3.0,
        'Marcos': 3.5,
        'Pedro': 4.0,
        'Claudia': 4.5,
        'Adriano': 3.0,
        'Janaina': 3.0
    }
}

from math import sqrt

# ==========================================================
# 1. Função para calcular similaridade pela distância Euclidiana
# ==========================================================
def euclidiana(base, usuario1, usuario2):
    """
    Calcula a similaridade entre dois usuários (ou itens) usando distância euclidiana.
    Retorna um valor entre 0 e 1, onde 1 é mais parecido.
    """
    # Dicionário para armazenar apenas filmes avaliados por ambos
    si = {}
    for item in base[usuario1]:
        if item in base[usuario2]:
            si[item] = 1

    # Se não há itens em comum, similaridade é zero
    if len(si) == 0:
        return 0

    # Soma dos quadrados das diferenças das notas
    soma = sum([
        pow(base[usuario1][item] - base[usuario2][item], 2)
        for item in base[usuario1] if item in base[usuario2]
    ])

    # Converte distância para similaridade (1 / (1 + distância))
    return 1 / (1 + sqrt(soma))

# ==========================================================
# 2. Função para encontrar os mais semelhantes a um usuário/item
# ==========================================================
def getSimilares(base, usuario):
    """
    Retorna lista dos mais semelhantes ao usuário/item dado.
    """
    similaridade = [
        (euclidiana(base, usuario, outro), outro)
        for outro in base if outro != usuario
    ]
    # Ordena do mais similar para o menos similar
    similaridade.sort()
    similaridade.reverse()
    return similaridade[0:30]  # limita a 30 resultados

# ==========================================================
# 3. Função para gerar recomendações personalizadas
# ==========================================================
def getRecomendacoesUsuario(base, usuario):
    """
    Recomenda itens para um usuário com base nas avaliações de outros usuários semelhantes.
    """
    totais = {}  # soma ponderada das notas
    somaSimilaridade = {}  # soma das similaridades

    for outro in base:
        if outro == usuario:
            continue
        similaridade = euclidiana(base, usuario, outro)

        for item in base[outro]:
            if item not in base[usuario]:
                totais.setdefault(item, 0)
                totais[item] += base[outro][item] * similaridade
                somaSimilaridade.setdefault(item, 0)
                somaSimilaridade[item] += similaridade

    # Calcula média ponderada das notas
    rankings = [
        (total / somaSimilaridade[item], item)
        for item, total in totais.items()
    ]
    rankings.sort()
    rankings.reverse()
    return rankings[0:30]

# ==========================================================
# 4. Função para calcular similaridade entre itens
# ==========================================================
def calculaItensSimilares(base):
    """
    Calcula e retorna dicionário com itens mais similares entre si.
    """
    result = {}
    for item in base:
        notas = getSimilares(base, item)
        result[item] = notas
    return result

# ==========================================================
# 5. Função para recomendações baseadas em itens
# ==========================================================
def getRecomendacoesItens(baseUsuario, similaridadeItens, usuario):
    """
    Gera recomendações usando similaridade entre itens (Item-Based Collaborative Filtering).
    """
    notasUsuario = baseUsuario[usuario]
    notas = {}
    totalSimilaridade = {}

    for (item, nota) in notasUsuario.items():
        for (similaridade, item2) in similaridadeItens[item]:
            if item2 in notasUsuario:
                continue
            notas.setdefault(item2, 0)
            notas[item2] += similaridade * nota
            totalSimilaridade.setdefault(item2, 0)
            totalSimilaridade[item2] += similaridade

    rankings = [
        (score / totalSimilaridade[item], item)
        for item, score in notas.items()
    ]
    rankings.sort()
    rankings.reverse()
    return rankings

# ==========================================================
# 6. Função para contar quantas avaliações cada filme recebeu
# ==========================================================
def count_evaluations(base: dict, filmes):
    """
    Conta quantos usuários avaliaram cada filme da lista.
    """
    result = []
    for filme in filmes:
        count = 0
        for usuario in base:
            if filme in base[usuario]:
                count += 1
        result.append({filme: count})
    return result

# ==========================================================
# Exemplo de execução
# ==========================================================
if __name__ == "__main__":
    lista_filmes = ['Freddy x Jason', 'O Ultimato Bourne', 'Star Trek', 'Exterminador do Futuro', 'Norbit', 'Star Wars']

    # Recomendações para um filme (usando base de filmes)
    print("Recomendações para 'Star Trek' (base de filmes):")
    print(getRecomendacoesUsuario(avaliacoesFilme, 'Star Trek'))

    # Recomendações para um usuário (usando base de usuários)
    print("\nRecomendações para Leonardo (base de usuários):")
    print(getRecomendacoesUsuario(avaliacoesUsuario, 'Leonardo'))

    # Contagem de avaliações
    print("\nContagem de avaliações por filme:")
    print(count_evaluations(avaliacoesUsuario, lista_filmes))


Recomendações para 'Star Trek' (base de filmes):
[(3.5891975244498413, 'Janaina'), (3.1853460676191814, 'Pedro'), (2.8079125834239766, 'Leonardo')]

Recomendações para Leonardo (base de usuários):
[(3.457128694491423, 'Star Wars'), (2.778584003814924, 'Freddy x Jason'), (2.422482042361917, 'Star Trek')]

Contagem de avaliações por filme:
[{'Freddy x Jason': 5}, {'O Ultimato Bourne': 7}, {'Star Trek': 4}, {'Exterminador do Futuro': 7}, {'Norbit': 6}, {'Star Wars': 6}]
